Data Preparation and Preprocessing

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
# Load and preprocess the data
base_dir = 'Food-101'

In [3]:
# ImageDataGenerator for data augmentation and normalization
data_gen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split the data into 80% training and 20% validation
)


In [4]:
# Create generators for training and validation
train_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 8080 images belonging to 101 classes.
Found 2020 images belonging to 101 classes.


In [5]:
# Test data generator (no data augmentation, just normalization)
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Ensure the order of images is preserved for evaluation
)

Found 10100 images belonging to 101 classes.


Model Selection

In [6]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [7]:
# Load the pretrained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [8]:
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(101, activation='softmax')(x)

In [9]:
# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

In [10]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10


C:\Users\sandh\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


253/253 ━━━━━━━━━━━━━━━━━━━━ 3207s 12s/step - accuracy: 0.0234 - loss: 4.6684 - val_accuracy: 0.0099 - val_loss: 5.5208
Epoch 2/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 2708s 11s/step - accuracy: 0.0508 - loss: 4.2651 - val_accuracy: 0.0099 - val_loss: 7.2612
Epoch 3/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 2845s 11s/step - accuracy: 0.1081 - loss: 3.8430 - val_accuracy: 0.0109 - val_loss: 6.6245
Epoch 4/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 3143s 12s/step - accuracy: 0.1600 - loss: 3.5218 - val_accuracy: 0.1045 - val_loss: 4.7746
Epoch 5/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 2769s 11s/step - accuracy: 0.1992 - loss: 3.2907 - val_accuracy: 0.1673 - val_loss: 3.6786
Epoch 6/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 2253s 9s/step - accuracy: 0.2335 - loss: 3.0883 - val_accuracy: 0.0941 - val_loss: 4.3836
Epoch 7/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 2749s 11s/step - accuracy: 0.2850 - loss: 2.8996 - val_accuracy: 0.2114 - val_loss: 3.3360
Epoch 8/10
253/253 ━━━━━━━━━━━━━━━━━━━━ 3072s 12s/step - accuracy: 0.3279 - loss: 2.6260 - val_

Saving the Model

In [12]:
import pickle

# Save the model to a file
model.save('food_recognition_model.h5')

# Save the label mapping
with open('label_mapping.pkl', 'wb') as f:
    pickle.dump(train_generator.class_indices, f)
